In [4]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import numpy as np
import pyarrow.parquet as pq
import torch.nn as nn
import torch.nn.functional as F

LOAD DATASET

In [7]:

class JointPointsDataset(Dataset):
    def __init__(self, parquet_file, transform=None):
        self.data = pq.read_table(parquet_file).to_pandas()
        self.transform=transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Obtenez les différentes informations de la ligne idx
        participant_id = self.data.iloc[idx, 1]
        exercise_type = self.data.iloc[idx, 2]
        camera_type = self.data.iloc[idx, 3]
        time = self.data.iloc[idx, 4]

        # Obtenez les coordonnées des joints du corps (à partir de la 6e colonne)
        joint_points = self.data.iloc[idx, 5:].values.astype(float)

        # Créez un dictionnaire avec toutes les informations
        sample = {
            'participant_id': participant_id,
            'exercise_type': exercise_type,
            'camera_type': camera_type,
            'time': time,
            'joint_points': joint_points
        }

        # Appliquez des transformations si nécessaire
        # Appliquez des transformations si nécessaire
        if self.transform:
            sample = self.transform(sample)

        return sample

In [8]:

# Créer une instance de votre dataset avec un fichier Parquet
dataset = JointPointsDataset(parquet_file='/Users/loufourneaux/Desktop/EPFL/MA1/ML/project2/All_Relative_Results_Cleaned.parquet')

# Créer un DataLoader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Utiliser le DataLoader pour itérer sur vos données pendant l'entraînement
for batch in dataloader:
    participant_ids = batch['participant_id']
    joint_points = batch['joint_points']

CREATE MODEL
- 2 layers de profondeur 8 
- avance avec la fonction relu 
- 99 input features (les joints et le temps)
- 7 output features = exercices 

In [ ]:
class Model(nn.Module):
    
    #input layer of network--> hidden layer 1--> hidden layer---> output layer (7 classes)
    def __init__(self, input_features=99, h1=8, h2=8, output_features=7):
        super().__init__()
        #moving from one layer to another
        self.fc1=nn.Linear(input_features,h1)
        self.fc2=nn.Linear(h1, h2)
        self.out=nn.Linear(h2, output_features)

    def forward(self, x): #need to actually move from layer to layer 
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)

        return x

TRAIN MODEL 
do i have to drop some columns when loading the data? 

In [ ]:
def modify_batch(batch):

    #change exercice column from strings to interger
    for batch in dataloader:
        batch['Exercise']= batch['Exercise'].replace('Stretch', 0)
    #... do this for all of them for classification